In [48]:
!pip install torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 5.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pydub
import librosa
import soundfile
import nemo.collections.asr as nemo_asr
import glob
import nemo.collections.tts as nemo_tts
import numpy as np
import torch
import shutil
import pyctcdecode
from word_beam_search import WordBeamSearch
from ctcdecode import CTCBeamDecoder
import itertools
import symspellpy
import matplotlib.pyplot as plt
from dataclasses import dataclass
import torchaudio
from forced_alignment import get_trellis, backtrack, plot_trellis_with_path, merge_repeats, plot_trellis_with_segments, merge_words
from IPython.display import Audio
import os

[NeMo W 2022-12-16 17:23:10 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-12-16 17:23:11 experimental:27] Module <class 'nemo_text_processing.g2p.modules.IPAG2P'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-12-16 17:23:11 experimental:27] Module <class 'nemo.collections.common.tokenizers.text_to_speech.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-12-16 17:23:11 experimental:27] Module <class 'nemo.collections.tts.models.radtts.RadTTSModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [23]:
!youtube-dl -f 140 https://www.youtube.com/watch\?v=h3HaNUXlzQc

[youtube] h3HaNUXlzQc: Downloading webpage
[download] Destination: Light vs. Medium vs. Dark Roasts Coffee _ What’s the difference-h3HaNUXlzQc.m4a
[download] 100% of 8.10MiB in 02:1349KiB/s ETA 00:000
[ffmpeg] Correcting container in "Light vs. Medium vs. Dark Roasts Coffee _ What’s the difference-h3HaNUXlzQc.m4a"


In [24]:
!ffmpeg -y -i *h3HaNUXlzQc.m4a -vn  -ac 1 -ar 16000  audioclips/h3HaNUXlzQc.wav
#!rm *uiDzZZoWkOI.mkv

ffmpeg version n5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 12.2.0 (GCC)
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libiec61883 --enable-libjack --enable-libmfx --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librav1e --enable-librsvg --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libv4l2 --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxcb --enable-libxml2 -

In [25]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model().to('cpu')
labels = bundle.get_labels()

In [26]:
sm = symspellpy.SymSpell(max_dictionary_edit_distance=3)
grammar_words = ['gingival', 'light',  'medium', 'mesial', 'lingual', 'distal', 'bleeding', 'furcation', 'plaque', 'margin', 'severe']

for w in grammar_words:
    sm.create_dictionary_entry(w, 1)

In [27]:
for audio_file in glob.glob("./audioclips/h3HaNUXlzQc.wav"):
    seg = pydub.AudioSegment.from_file(audio_file)
    filename = audio_file.split('/')[-1].split('.')[0]
    print(filename)
    segs = pydub.silence.split_on_silence(seg, min_silence_len=1000, silence_thresh=-40)
    try:
        os.mkdir("./tmp/")
    except: 
        print("exists")
        pass
    for i, seg in enumerate(segs):
        pydub.AudioSegment.export(seg, f"tmp/{i}.wav", format='wav')

    for f in glob.glob('tmp/*.wav')[1:]:
        y, _ = librosa.load(f, sr=16_000)
        if len(y) > 16_000*45: #Too long to fit in RAM
            continue
        emissions, _ = model(torch.Tensor([y]))
        transcript = [labels[i] for i in np.argmax(emissions[0].detach().numpy(), axis=1)]
        transcript = [k for (k, g) in itertools.groupby(transcript)]
        transcript = ''.join(transcript).replace('-','')
        tmp = [sm.lookup_compound(w, 1)[0].term.upper() if len(sm.lookup(w.lower(), 1)) else w for w in transcript.split('|')]
        transcript = "|".join(tmp)
        if len(transcript.split('|')) <= 1:
            continue
        dictionary = {c: i for i, c in enumerate(labels)}
        tokens = [dictionary[c] for c in transcript]
        trellis = get_trellis(emissions[0], tokens)
        path = backtrack(trellis, emissions[0], tokens)
        segments = merge_repeats(path, transcript)
        word_segments = merge_words(segments)
        for word in word_segments:
            for grammar_word in grammar_words:
                if word.label == grammar_word.upper():
                    #print(word)
                    try:
                        os.mkdir(f"./train_samples/{grammar_word}")
                    except: 
                        pass
                        #print("exists")
                    i = len(glob.glob(f'./train_samples/{grammar_word}/{filename}_*'))
                    l = len(y)
                    start = word.start/emissions.shape[1]
                    end = word.end/emissions.shape[1]
                    st = int(start*l)
                    end =int(end*l)
                    soundfile.write(f"./train_samples/{grammar_word}/{filename}_{i}.wav", y[st:end], samplerate=16_000)
    shutil.rmtree('./tmp')

h3HaNUXlzQc
